# **Prepare Data**

In [38]:
import pandas as pd
import numpy as np


In [39]:
df = pd.read_csv("employee_training_data.csv")
df

,department_0,department_1,department_2,department_3,department_4,department_5,department_6,department_7,department_8,department_9,promoted,review,projects,salary,tenure,satisfaction,bonus,avg_hrs_month,left
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,3,0,5.0,4.0,0,0.0,0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,4,1,7.0,2.0,0,1.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,4.0,4,1,5.0,0.0,0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,4.0,3,1,9.0,2.0,0,2.0,0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0,3,1,5.0,2.0,0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9527,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4.0,4,1,6.0,1.0,0,1.0,1
9528,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0,3,1,6.0,1.0,1,0.0,1
9529,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,3,0,7.0,4.0,0,1.0,1
9530,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4.0,4,2,4.0,1.0,0,0.0,1


In [40]:
test_df = pd.read_csv("employee_test_data.csv")
test_df

,department_0,department_1,department_2,department_3,department_4,department_5,department_6,department_7,department_8,department_9,promoted,review,projects,salary,tenure,satisfaction,bonus,avg_hrs_month,left
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.824216,3,1,5.0,0.371939,1,179.452033,1
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.610875,3,2,5.0,0.529254,0,178.823881,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.532324,4,1,8.0,0.454286,0,188.124078,0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.708532,3,1,10.0,0.549473,0,195.349963,0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.800551,3,1,8.0,0.648343,0,188.612317,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2857,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.687485,4,1,5.0,0.752645,0,180.121531,1
2858,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.836415,3,1,4.0,0.328842,0,177.723585,1
2859,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.411028,3,1,8.0,0.393935,0,190.368509,0
2860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.615693,4,2,5.0,0.336712,0,181.492881,0


In [41]:
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score,KFold

In [42]:
X_train = df.loc[:,df.columns != 'left']
y_train = df['left']
X_test = test_df.loc[:,df.columns != 'left']
y_test = test_df['left']

# X_train ,X_test,y_train,y_test  = train_test_split(X,y,test_size=0.3,random_state=0)




In [57]:
for i in X_train:
    print(X_train[i].unique())

[0.]
[0. 1.]
[1. 0.]
[0.]
[0.]
[0. 1.]
[0.]
[0. 1.]
[0. 1.]
[0.]
[0 1]
[1. 4. 3. 2. 0.]
[3 4 2 5]
[0 1 2]
[ 5.  7.  9.  8.  4.  6.  3. 11. 10.  2.]
[4. 2. 0. 3. 1.]
[0 1]
[0. 1. 2.]


# **Voting Classifier**

In [44]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

## **Default parameters**

In [45]:
clf1 = KNeighborsClassifier() 
clf2 = SVC()  


voting_clf = VotingClassifier(
    estimators=[('SVC', clf2), ('KNeighborsClassifier', clf1)],
    voting='hard'
    )

score = cross_val_score(voting_clf, X_train, y_train, cv=5)
print("train score : ", score.mean())

voting_clf.fit(X_train, y_train)
print("test score : ", voting_clf.score(X_test, y_test))

train score :  0.7866158313299816
test score :  0.6953179594689028


## **With hyper parameters**

In [46]:
import warnings
warnings.filterwarnings('ignore')

In [47]:
grid = {
    # "n_neighbors": [5, 10, 20],
    "n_neighbors": [20],
    "algorithm": ['auto'],
    # "leaf_size": [10, 20, 30, 40, 50],
    "leaf_size": [10],
    # "p": [0.01, 0.1, 1, 2, 10, 100],
    "p": [0.01, 0.1],
}
gcv_knb = GridSearchCV(KNeighborsClassifier(), grid)

grid = {
    "gamma" : ["scale","auto"],
}

gcv_svc = GridSearchCV(SVC(),grid)

In [48]:
voting_clf = VotingClassifier(
    estimators=[('gcv_svc', gcv_svc), ('gcv_knb', gcv_knb)],
    voting='hard'
    )

score = cross_val_score(voting_clf, X_train, y_train, cv=5)

print("train score : ", score.mean())
voting_clf.fit(X_train, y_train)
print("test score : ", voting_clf.score(X_test, y_test))

train score :  0.7973148575607293
test score :  0.6953179594689028


# **Stacking Classifier**

In [49]:
from sklearn.ensemble import StackingClassifier

## **default parameters**

In [50]:
stack_clf = StackingClassifier(
    estimators=[('SVC', SVC()), ('KNeighborsClassifier', KNeighborsClassifier())],
    stack_method='predict',
    cv=10
    )

score = cross_val_score(stack_clf, X_train, y_train, cv=5)

print("train score : ", score.mean())
stack_clf.fit(X_train, y_train)
print("test score : ", stack_clf.score(X_test, y_test))

train score :  0.8150480006558924
test score :  0.6397624039133473


## **With hyper parameter**

In [51]:
grid = {
    # "n_neighbors": [5, 10, 20],
    "n_neighbors": [20],
    "algorithm": ['auto'],
    # "leaf_size": [10, 20, 30, 40, 50],
    "leaf_size": [10],
    # "p": [0.01, 0.1, 1, 2, 10, 100],
    "p": [0.01, 0.1],
}
gcv_knb = GridSearchCV(KNeighborsClassifier(), grid)

grid = {
    "gamma" : ["scale","auto"],
}

gcv_svc = GridSearchCV(SVC(),grid)

In [52]:
stack_clf = StackingClassifier(
    estimators=[('gcv_svc', gcv_svc), ('gcv_knb', gcv_knb)],
    stack_method='auto'
    )

score = cross_val_score(stack_clf, X_train, y_train, cv=5)

print("train score : ", score.mean())
stack_clf.fit(X_train, y_train)
print("test score : ", stack_clf.score(X_test, y_test))

train score :  0.8049722373692548
test score :  0.6733053808525507


# **Baseline** 

In [53]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(X_train, y_train)
dummy_clf.score(X_train, y_train)


0.5

In [54]:
dummy_clf = DummyClassifier(strategy='prior')
dummy_clf.fit(X_train, y_train)
dummy_clf.score(X_train, y_train)

0.5

In [55]:
dummy_clf = DummyClassifier(strategy='stratified')
dummy_clf.fit(X_train, y_train)
dummy_clf.score(X_train, y_train)

0.5008392782207302

In [56]:
dummy_clf = DummyClassifier(strategy='uniform')
dummy_clf.fit(X_train, y_train)
dummy_clf.score(X_train, y_train)

0.5065044062106588